# <center>**Elliptic Curve Cryptography**<center/>
---

## What is an Elliptic Curve?

### Elliptic Curve Fundamentals

An elliptic curve $E$ over a field $F$ is a collection of points $(x, y)$ where $x,y\in F$ satisfying the following equation:
$$y^2=x^3+ax+b$$
and containing the "point at infinity" $O$.

#### Identitites and Inverses

The point at infinity is the additive identity. That is, for all points $P\in F$, $P+O=P$

Let us also say that $P=(x, y)$. Then we will define $-P=(x,-y)$. We then say that $P+(-P)=O$

#### Addition

We define addition for two distinct points $P$ and $Q$ as $P+Q$ on an elliptic curve by first calculating the secant line between $P$ and $Q$. We then follow that line until it next intersects the curve. Let us call this point $J$. We then say that $P+Q=-J$. A graphical interpretation of addition is shown below:

<br/><br/>
<img src="images/addition_distinct.png" alt="Addition along an Elliptic Curve">
<br/><br/>

For point doubling ($2P=P+P$) we perform the same process but now with line tangent to the elliptic curve at point $P$ instead of the secant line.

### Elliptic Curves over Finite Fields

In the examples and images we looked at above, all calculations were done over the field of the real numbers, $\mathbb{R}$. However, for our purposes we will work exclusively with elliptic curves over the field $\mathbb{Z}/p\mathbb{Z}$ (the integers modulo a prime $p$). This means our elliptic curve equation above will now look like this:
$$y^2=x^3+ax+b\mod{p}$$
These finite fields are much easier for computers to work with, but are much less intuitive. Here is an example of an elliptic curve over a finite field:

<br/><br/>
<img src="images/finite_ec.png" alt="Elliptic Curve over finite field">
<br/><br/>

### **Notice how not every x-coordinate contains a point on the elliptic curve.**

### Testing Elliptic Curves Out

An elliptic curve implementation is available in the file `ec/elliptic_curve.py`. Let's test it out:

In [7]:
from ec.elliptic_curve import EllipticCurve

In [8]:
# Define our Elliptic Curve
ecurve = EllipticCurve(0, 3, 23) # x^3 + 3x (mod 23)

# Define two of our points
p = (12, 11)
q = (1, 2)

# Print results from operations on Elliptic Curves with our points
print("P + O =", ecurve.add_points(p, ecurve.PTATINF)) # P + O
print("P + (Q - P) =", ecurve.add_points(p, ecurve.add_points(q, ecurve.neg(p)))) # P + (Q - P) should give Q


P + O = (12, 11)
P + (Q - P) = (1, 2)


## Uses of an Elliptic Curve

There are many uses for elliptic curves and their math in cryptography, we'll go over the two that we chose to implement here: Elliptic Curve Diffie-Helmann and an Elliptic Curve El Gamal encryption-decryption system.

### Elliptic Curve Diffie-Helmann

The idea behind this is quite simple. Alice and Bob agree upon a curve $E$ and a point $P$ on that curve. Then they each choose a scalar private key which well call $a$ and $b$, respectively. Alice and Bob each compute their respective public keys $aP$ and $bP$, which are both points on the elliptic curve. Then Alice takes Bob's public key and forms the point $a(bP)$ while Bob uses Alice's public key to form the point $b(aP)$. Because the points on an elliptic curve form a group, $a(bP)=b(aP)$. This means that Alice and Bob will both end up with the same shared key without ever knowing the other person's private key.

Let's look at an example using our libraries. It is important to note that the `ECDH` classes randomly select a private key, which means this code is nondeterministic.

In [11]:
from ecdh.ecdh import ECDH

In [12]:
new_ecurve = EllipticCurve(0, 3, 19) # y^2 = x^3 + 3 (mod 19)
p = (1, 2)

# for i in range(19):
#     print(i, new_ecurve.scalar_product(i, p))

alice = ECDH(p, new_ecurve)
bob = ECDH(p, new_ecurve)

a_pub = alice.get_public()
b_pub = bob.get_public()

print("Alice's public key: ", a_pub)
print("Bob's public key: ", b_pub)

print("Alice's shared key: ", alice.get_shared(b_pub))
print("Bob's shared key: ", bob.get_shared(a_pub))

Alice's public key:  (0, 0)
Bob's public key:  (2, 7)
Alice's shared key:  (0, 0)
Bob's shared key:  (0, 0)


### Elliptic Curve El-Gamal Cryptosystem

We start with an agreed upon elliptic curve $E$ and an agreed upon point $P$ with order $n$. Alice wishes to encrypt a point $T$ and send it to Bob who has public key $B$. Alice chooses an integer $a\in[0,n-1]$. Alice will then send a pair of points $(A, Y)$ where $A=a\cdot P$ (Alice's public key) and $Y=T+a\cdot B$ (point to encrypt + shared key).

Then Bob will receive a pair of points $(A, Y)$. He has also already chosen an integer $b\in[0, n-1]$ when he calculated $B=b\cdot P$ (his public key). He will then calculate $T=Y-b\cdot A$ (encrypted point - shared key), undoing the encryption operation Alice performed.

In [14]:
from ec_elgamal.ec_actor import ECActor

In [15]:
alice = ECActor(p, 13, new_ecurve) # 13p = O (point at infinity)
bob = ECActor(p, 13, new_ecurve)

# Calculate public keys for Alice and Bob
a_pub = alice.get_public()
b_pub = bob.get_public()

# Define the point we want to encrypt
to_encrypt = (11, 17)

# Have Alice encrypt the point
alice_encrypted = alice.encrypt(to_encrypt, b_pub)

# Have Bob encrypt the point
bob_encrypted = bob.encrypt(to_encrypt, a_pub)

# Print encrypted values
print("Alice's encrypted pair:", alice_encrypted)
print("Bob's encrypted pair:", bob_encrypted)

# Have Alice decrypt Bob's Encrypted values
alice_decrypted = alice.decrypt(bob_encrypted)

# Have Bob decrypt Alice's Encrypted values
bob_decrypted = bob.decrypt(alice_encrypted)

# Print decrypted values
print("Alice's decrypted value:", alice_decrypted)
print("Bob's decrypted value:", bob_decrypted)

Alice's encrypted pair: ((1, 17), (0, 0))
Bob's encrypted pair: ((11, 17), (0, 0))
Alice's decrypted value: (11, 17)
Bob's decrypted value: (11, 17)


## Implementing a Real Elliptic Curve

Elliptic Curves and Elliptic Curve Cryptography is used widely in many situations requiring assymetric key exchanges, due to its advantages over RSA. These advantages include but are not limited to:

- Less memory usage (smaller keys + key compression)
- Faster
- More secure

One common usage of elliptic curves is in blockchain, where lots of encryption/decryption operations need to be made extremely quickly. Bitcoin, for example, uses a specific elliptic curve known as `Secp256k1`, defined as $y^2=x^3+7\mod{p}$ where $p$ is an extremely large prime. Below, we create this curve using our defined classes.

In [17]:
# Define Secp256k1, the elliptic curve that Bitcoin uses. Parameters found here: https://en.bitcoin.it/wiki/Secp256k1

p = 0xFFFFFFFF_FFFFFFFF_FFFFFFFF_FFFFFFFF_FFFFFFFF_FFFFFFFF_FFFFFFFE_FFFFFC2F

ecurve = EllipticCurve(0, 7, p) # y^2 = x^2 + 7 (mod p)

Implementations of Secp256k1 also usually use an agreed-upon generator point `g`, which is defined below. This point also has a predefined order `ord_g`

In [19]:
# Define a generator point g and its order

g = (55066263022277343669578718895168534326250603453777594175500187360389116729240,
     32670510020758816978083085130507043184471273380659243275938904335757337482424)

ord_g = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

Now we will implement our Diffie-Hellman system using this curve and this point:

In [37]:
# Define actors alice and bob
alice = ECDH(g, ecurve)
bob = ECDH(g, ecurve)

# Calculate respective public keys for alice and bob
a_pub = alice.get_public()
b_pub = bob.get_public()

# Display their public keys
print("Alice public:", a_pub)
print("Bob public:", b_pub)
print()

# Calculate respective shared keys
a_share = alice.get_shared(b_pub)
b_share = bob.get_shared(a_pub)
print()

# Display their shared keys
print("Alice shared:", alice.get_shared(b_pub))
print("Bob shared:", bob.get_shared(a_pub))
print()

# Are their keys equal?
print("Are Alice and Bob's shared keys equal:", a_share == b_share)

Alice public: (13070943797601888935180029660785623989587668759438828966193335861568978001100, 103698213196095570860588193975803109523779010114271451511650815967763016335350)
Bob public: (40827429754020400277641792070834887950857537740917686941843149402637679790485, 56057296843397433440323752630575952606553615946307492115777841665982348839894)


Alice shared: (48211267323708301775355951149360211370071505300026048838605265909626530451860, 2995626562847617694677804900330792343079397876662666010528698592822448048572)
Bob shared: (48211267323708301775355951149360211370071505300026048838605265909626530451860, 2995626562847617694677804900330792343079397876662666010528698592822448048572)

Are Alice and Bob's shared keys equal: True


Now we will implement our El Gamal system on this curve, using the `EllipticCurve.random_point(self)` method to encrypt random points along our curve:

In [35]:
# Define our actors for our system using the generator point previously defined
alice_gamal = ECActor(g, ord_g, ecurve)
bob_gamal = ECActor(g, ord_g, ecurve)

# Calculate their respective public keys
a_pub = alice_gamal.get_public()
b_pub = bob_gamal.get_public()

# Display public keys
print("Alice's public:", a_pub)
print("Bob's public:", b_pub)
print()

# Generate and display a random point along the curve
to_encrypt = ecurve.random_point()
print("Encrypting: ", to_encrypt)
print()

# Have each of our actors encrypt our random point
alice_encrypt = alice_gamal.encrypt(to_encrypt, b_pub)
bob_encrypt = bob_gamal.encrypt(to_encrypt, a_pub)

# Display the encrypted values
print("Alice encrypted:", alice_encrypt)
print("Bob encrypted:", bob_encrypt)
print()

# Decrypt the other actors' encrypted values
alice_decrypt = alice_gamal.decrypt(bob_encrypt)
bob_decrypt = bob_gamal.decrypt(alice_encrypt)

# Display decrypted values
print("Alice decrypted: ", alice_decrypt)
print("Bob decrypted: ", bob_decrypt)
print()

# Assert that all the encrypted values are the same
print("Decrypted values match values to encrypt?", to_encrypt == alice_decrypt == bob_decrypt)

Alice's public: (51561966352142705708272210901373919291584252647338363608285546303667474680344, 23511015351317386827141164473818490604928871527344169028869271178219834331127)
Bob's public: (112829080582655768297170033809668095921189158631638553439117119254019441557524, 88609633792240659368318852647002060616221516476581875348400556556638598968560)

Encrypting:  (74307979097536844793952285383890265841890098611954712669327047519382507046136, 69536337843899626336336776461237098341930503583734513770833513048016224557040)

Alice encrypted: ((51561966352142705708272210901373919291584252647338363608285546303667474680344, 23511015351317386827141164473818490604928871527344169028869271178219834331127), (1349492574593667667220636259109735049712205758576162522580341531603442300968, 105344402465303331915164095720118567412300902979566742563487415315603666629055))
Bob encrypted: ((112829080582655768297170033809668095921189158631638553439117119254019441557524, 886096337922406593683188526470020606162215